In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
base_dir = ".."

In [ ]:
all_plddts = json.loads(open(f"{base_dir}/data/pfam_plddt.json").read())
conserved_residues = json.loads(open(f"{base_dir}/tmp/residue_features/conserved.json").read())

In [ ]:
def avg(num_list):
    return sum(num_list)/len(num_list)

In [ ]:
cons_minus_bg_plddt = {}
for seed_id, con_res in conserved_residues.items():
    if len(con_res) == 0:
        continue
    bkg_plddt = all_plddts[seed_id]
    bkg_avg_plddt = avg(bkg_plddt)
    conserved_residues_avg_plddt = avg([bkg_plddt[i-1] for i in con_res])
    cons_minus_bg_plddt[seed_id] = conserved_residues_avg_plddt - bkg_avg_plddt

In [ ]:
data = pd.DataFrame({ "seed_id": cons_minus_bg_plddt.keys(), "conserved_residue_plddt_minus_bkg": cons_minus_bg_plddt.values()})
sorted_data = data.sort_values(by="conserved_residue_plddt_minus_bkg" ,ascending = False).reset_index(drop=True)

In [ ]:
more_confident_than_background = (sorted_data["conserved_residue_plddt_minus_bkg"] > 0).argmin() / len(sorted_data)

In [ ]:
print(f"For {round(100 * more_confident_than_background, 2)} percent of the seeds, the pLDDT of the conserved sites is higher than the rest of the seed")

In [ ]:
seeds_with_low_confidence_conserved_sites = sorted_data[sorted_data["conserved_residue_plddt_minus_bkg"]<0][["seed_id"]]

In [ ]:
seeds_with_low_confidence_conserved_sites.to_csv(f"{base_dir}/data/processed/seeds_with_low_confidence_conserved_sites.tsv", sep="\t", header=None, index=None)